In [7]:
from lib.generator import generate_binary_signal, plot_binary_signal
from lib.channel import apply_channel
from lib.receiver import sample_and_detect_nrz
from lib.ui import create_channel_ui, create_generator_ui, create_receiver_ui
import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np


signal_state = {}



ImportError: cannot import name 'create_receiver_ui' from 'lib.ui' (/home/kuribam/PycharmProjects/signal-transmission-simulator/lib/ui.py)

In [ ]:
def interactive_generator(Rb, frame_length, seed):
    bits, nrz, t, fs, Rb_used = generate_binary_signal(
        Rb=int(Rb),
        frame_length=int(frame_length),
        seed=int(seed)
    )

    # aktualizace sdíleného stavu
    signal_state["bits"] = bits
    signal_state["nrz"] = nrz
    signal_state["t"] = t
    signal_state["fs"] = fs
    signal_state["Rb"] = Rb_used

    plot_binary_signal(bits, nrz, t, Rb_used)

In [ ]:
gen_ui, gen_controls = create_generator_ui()

gen_out = widgets.interactive_output(
    interactive_generator,
    gen_controls
)

display(gen_ui, gen_out)


In [ ]:
def interactive_channel(B, snr_db, awgn_on, zoom_bits, show_noise):
    if "nrz" not in signal_state:
        print("⚠️ Nejprve vygeneruj signál")
        return

    nrz = signal_state["nrz"]
    t = signal_state["t"]
    fs = signal_state["fs"]
    Rb = signal_state["Rb"]

    channel_out = apply_channel(
        signal=nrz,
        fs=fs,
        B=float(B),
        snr_db=float(snr_db),
        awgn_on=awgn_on
    )

    samples_per_bit = int(fs / Rb)
    N = zoom_bits * samples_per_bit

    plt.figure(figsize=(12, 4))
    plt.step(t[:N], nrz[:N], where="post", alpha=0.4, label="NRZ")
    plt.step(t[:N], channel_out[:N], where="post", label="Po kanálu")

    if awgn_on and show_noise:
        noise = channel_out - nrz
        plt.step(t[:N], noise[:N], where="post", label="Šum")

    plt.xlabel("Čas [s]")
    plt.ylabel("Amplituda")
    plt.title("Kanál: LPF + AWGN")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

    signal_state["channel_out"] = channel_out




In [ ]:
channel_ui, channel_controls = create_channel_ui()

channel_out = widgets.interactive_output(
    interactive_channel,
    channel_controls
)

display(channel_ui, channel_out)



In [ ]:
def interactive_receiver(threshold, show_samples):
    if "nrz" not in signal_state:
        print("⚠️ Nejprve vygeneruj signál a aplikuj kanál")
        return

    nrz = signal_state["nrz"]
    t = signal_state["t"]
    fs = signal_state["fs"]
    Rb = signal_state["Rb"]

    # použij poslední výstup kanálu
    signal_rx = signal_state.get("channel_out", nrz)

    bits_rx, samples = sample_and_detect_nrz(
        signal_rx,
        fs,
        Rb,
        threshold=threshold
    )

    samples_per_bit = int(fs / Rb)
    N = samples_per_bit * 10  # 10 bitů na zobrazení

    plt.figure(figsize=(12, 4))
    plt.step(t[:N], signal_rx[:N], where="post", label="Signál po kanálu")

    if show_samples:
        sample_times = t[
            np.arange(samples_per_bit // 2, len(signal_rx), samples_per_bit)
        ]
        plt.plot(sample_times[:10], samples[:10], "ro", label="Vzorky")

    plt.axhline(threshold, color="k", linestyle="--", label="Práh")

    plt.xlabel("Čas [s]")
    plt.ylabel("Amplituda")
    plt.title("Receiver: vzorkování a detekce NRZ")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
rx_ui, rx_controls = create_receiver_ui()

rx_out = widgets.interactive_output(
    interactive_receiver,
    rx_controls
)

display(rx_ui, rx_out)
